In [2]:
import json
with open("filtered_lowcap_stocks.json", "r") as f:
    filtered = json.load(f)


In [3]:
def get_env():
    with open("../.env") as f:
        lines = f.readlines()
        env = {}
        for line in lines:
            if "=" in line:
                k, v = line.strip().replace('"', '').split("=", 1)
                env[k] = v
    return env
env = get_env()


In [4]:
from __future__ import annotations

import math
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

def chunked(xs, n):
    for i in range(0, len(xs), n):
        yield xs[i:i+n]

def alpaca_liq_vol_screen(
    symbols: list[str],
    auth: dict,
    days: int = 180,
    bar_timeframe: str = "1Day",
    symbols_per_call: int = 200,
    sleep_s: float = 0.25,
    adv_min: float = 150_000.0,
    adv_max: float = 20_000_000.0,   # "too liquid / hedge-fund friendly" cutoff
    vol_window_short: int = 20,
    vol_window_long: int = 60,
    big_move: float = 0.15,          # 15% daily move threshold
    min_days_required: int = 90,
) -> pd.DataFrame:
    base = (auth.get("ALPACA_DATA_BASE_URL") or "https://data.alpaca.markets").rstrip("/")
    key_id = (auth.get("ALPACA_KEY") or "").strip().strip('"').strip("'")
    secret = (auth.get("ALPACA_SECRET") or "").strip().strip('"').strip("'")
    if not key_id or not secret:
        raise ValueError("Missing ALPACA_KEY / ALPACA_SECRET in auth")

    headers = {"APCA-API-KEY-ID": key_id, "APCA-API-SECRET-KEY": secret}

    end = datetime.now(timezone.utc)
    start = end - timedelta(days=days)

    all_rows = []

    for batch in chunked([s.upper() for s in symbols], symbols_per_call):
        params = {
            "symbols": ",".join(batch),
            "timeframe": bar_timeframe,
            "start": start.isoformat(),
            "end": end.isoformat(),
            "adjustment": "all",
            "feed": auth.get("ALPACA_DATA_FEED", "iex"),
            "limit": 10000,
        }
        r = requests.get(f"{base}/v2/stocks/bars", headers=headers, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()

        bars_by_symbol = data.get("bars", {})
        for sym, bars in bars_by_symbol.items():
            if not bars:
                continue
            df = pd.DataFrame(bars)
            # Alpaca fields: t,o,h,l,c,v,n,vw (t timestamp)
            if "t" not in df.columns or "c" not in df.columns or "v" not in df.columns:
                continue
            df["t"] = pd.to_datetime(df["t"], utc=True)
            df = df.sort_values("t")
            df["sym"] = sym
            all_rows.append(df[["sym", "t", "c", "v"]])

        time.sleep(sleep_s)

    if not all_rows:
        return pd.DataFrame(columns=[
            "symbol","n_days","last_close","adv20","adv60","vol20","vol60",
            "pct_big_moves_60d","max_dd_120d","liq_ok","hf_ok","data_ok","score"
        ])

    bars = pd.concat(all_rows, ignore_index=True)
    bars = bars.dropna(subset=["c", "v"])
    bars["dvol"] = bars["c"].astype(float) * bars["v"].astype(float)

    out = []
    for sym, g in bars.groupby("sym", sort=False):
        g = g.sort_values("t")
        n = len(g)
        if n < min_days_required:
            out.append({"symbol": sym, "n_days": n, "data_ok": False})
            continue

        close = g["c"].astype(float).to_numpy()
        dvol = g["dvol"].astype(float).to_numpy()

        rets = np.diff(np.log(close))
        # align return-based measures to last rows
        def roll_std(x, w):
            if len(x) < w:
                return np.nan
            return float(np.std(x[-w:], ddof=1))

        vol20 = roll_std(rets, vol_window_short) * math.sqrt(252)
        vol60 = roll_std(rets, vol_window_long) * math.sqrt(252)

        adv20 = float(np.mean(dvol[-min(vol_window_short, len(dvol)):]))
        adv60 = float(np.mean(dvol[-min(vol_window_long, len(dvol)):]))
        last_close = float(close[-1])

        # big-move frequency over last 60 trading days
        last_rets = rets[-min(vol_window_long, len(rets)):]
        pct_big = float(np.mean(np.abs(np.expm1(last_rets)) >= big_move)) if len(last_rets) else np.nan

        # max drawdown over last 120 trading days
        win = min(120, len(close))
        px = close[-win:]
        peak = np.maximum.accumulate(px)
        dd = (px / peak) - 1.0
        max_dd = float(dd.min())

        liq_ok = (adv20 >= adv_min)
        hf_ok = (adv20 <= adv_max)  # if False => "too liquid / likely crowded"
        data_ok = True

        # simple score: prefer liquid-enough but not-too-liquid, penalize extreme vol + jumpiness
        # (you can tune weights)
        score = 0.0
        if data_ok:
            # center liquidity around geometric mid of [adv_min, adv_max]
            mid = math.sqrt(adv_min * adv_max)
            score += -abs(math.log(max(adv20, 1.0) / mid))
            if not liq_ok:
                score -= 2.0
            if not hf_ok:
                score -= 1.5
            if not np.isnan(vol60):
                score -= 0.5 * max(0.0, (vol60 - 0.8))  # penalize >80% ann vol
            if not np.isnan(pct_big):
                score -= 2.0 * pct_big               # penalize jumpy names
            score += 0.25 * max_dd                   # drawdown is negative; bigger drawdown => lower score

        out.append({
            "symbol": sym,
            "n_days": n,
            "last_close": last_close,
            "adv20": adv20,
            "adv60": adv60,
            "vol20": vol20,
            "vol60": vol60,
            "pct_big_moves_60d": pct_big,
            "max_dd_120d": max_dd,
            "liq_ok": liq_ok,
            "hf_ok": hf_ok,
            "data_ok": data_ok,
            "score": score,
        })

    df_out = pd.DataFrame(out)

    # Fill missing columns for short-history rows
    for col in ["last_close","adv20","adv60","vol20","vol60","pct_big_moves_60d","max_dd_120d","liq_ok","hf_ok","score"]:
        if col not in df_out.columns:
            df_out[col] = np.nan

    # Rank: keep only data_ok, then liq_ok, then hf_ok, then score
    df_out = df_out.sort_values(
        by=["data_ok","liq_ok","hf_ok","score"],
        ascending=[False, False, False, False],
        kind="mergesort"
    ).reset_index(drop=True)

    return df_out


In [5]:
screen = alpaca_liq_vol_screen(
    symbols=list(filtered.keys()),   # your 213 symbols
    auth=env,
    days=365,
    adv_min=200_000,
    adv_max=20_000_000,
    big_move=0.15,
    symbols_per_call=50,
)


In [6]:
screen.head(10)

,symbol,n_days,last_close,adv20,adv60,vol20,vol60,pct_big_moves_60d,max_dd_120d,liq_ok,hf_ok,data_ok,score
0,DUST,250,5.840,3.541780e+06,3.134439e+06,0.703097,0.887591,0.016667,-0.759868,True,True,True,-0.838578
1,ELME,250,2.870,6.564464e+05,3.963512e+05,0.226579,0.152931,0.000000,-0.059595,True,True,True,-1.128960
2,UNG,250,11.320,5.708852e+06,4.604550e+06,0.722811,0.622602,0.000000,-0.364580,True,True,True,-1.140016
3,SPCE,250,3.080,7.050816e+05,5.257106e+05,0.648899,0.805177,0.016667,-0.344156,True,True,True,-1.164550
4,FCEL,250,7.520,6.941155e+05,7.997527e+05,1.146521,1.039477,0.033333,-0.475503,True,True,True,-1.363545
5,SITC,250,6.500,5.111240e+05,5.012912e+05,0.190387,0.243997,0.000000,-0.183844,True,True,True,-1.410251
6,PTLO,250,5.200,5.536657e+05,6.115804e+05,0.490735,0.505632,0.000000,-0.575686,True,True,True,-1.428263
7,RC,250,2.075,5.153672e+05,4.319559e+05,0.419840,0.500689,0.000000,-0.536501,True,True,True,-1.490148
8,ISSC,250,19.310,5.406963e+05,2.828315e+05,1.197928,0.808118,0.033333,-0.578105,True,True,True,-1.523297
9,FC,250,19.375,4.250869e+05,3.060991e+05,0.328294,0.479614,0.000000,-0.304369,True,True,True,-1.624701


In [ ]:
def get_json(url: str, headers : dict):
    r = requests.get(url, headers=headers, timeout = 30)
    r.raise_for_status()
    return r.json()
def extract_data(data: dict) -> dict:
    def safe_get(path: list, default=None):
        cur = data
        for p in path:
            if not isinstance(cur, dict) or p not in cur:
                return default
            cur = cur[p]
        return cur

    assets = []
    for item in safe_get(['facts','us-gaap','Assets','units','USD'], []):
        assets.append((item.get('end'), item.get('val')))

    liabilities = []
    for item in safe_get(['facts','us-gaap','Liabilities','units','USD'], []):
        liabilities.append((item.get('end'), item.get('val')))

    cash = []
    for item in safe_get(['facts','us-gaap','CashAndCashEquivalentsAtCarryingValue','units','USD'], []):
        cash.append((item.get('end'), item.get('val')))

    shares = []
    for item in safe_get(['facts','dei','EntityCommonStockSharesOutstanding','units','shares'], []):
        shares.append((item.get('end'), item.get('val')))

    net_income = []
    for item in safe_get(
        ['facts','us-gaap','NetIncomeLossAvailableToCommonStockholdersBasic','units','USD'],
        []
    ):
        net_income.append((item.get('end'), item.get('val')))

    revenue = []
    for item in safe_get(['facts','us-gaap','Revenues','units','USD'], []):
        revenue.append((item.get('end'), item.get('val')))

    ebit = []
    for item in safe_get(['facts','us-gaap','OperatingIncomeLoss','units','USD'], []):
        ebit.append((item.get('end'), item.get('val')))

    operatingcashflow = []
    for item in safe_get(
        ['facts','us-gaap','NetCashProvidedByUsedInOperatingActivities','units','USD'],
        []
    ):
        operatingcashflow.append((item.get('end'), item.get('val')))

    debt = []
    for item in safe_get(['facts','us-gaap','LongTermDebt','units','USD'], []):
        debt.append((item.get('end'), item.get('val')))

    return {
        "assets": assets,
        "liabilities": liabilities,
        "cash": cash,
        "shares": shares,
        "net_income": net_income,
        "revenue": revenue,
        "ebit": ebit,
        "operating_cash_flow": operatingcashflow,
        "long_term_debt": debt,
    }

def get_edgar(cik):
    COMPANYFACTS_URL_TMPL = "https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    headers = {"User-Agent": "leo@gmail.com", "Accept-Encoding": "gzip, deflate"}
    url = COMPANYFACTS_URL_TMPL.format(cik=cik)
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    return r.json()

def get_more_data(tickers):
    SEC_TICKER_MAP_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
    headers = {"User-Agent": "leo@gmail.com", "Accept-Encoding": "gzip, deflate"}    
    CIK= get_json(SEC_TICKER_MAP_URL, headers)
    out = {}
    print(f"Processing {len(tickers)} CIK data...")
    CIK_data = CIK['data']
    counter = 0
    for item in CIK_data:
        
        if item[2].upper() in tickers:
            CIK_code = str(item[0]).zfill(10) 
            try:
                data = get_edgar(CIK_code)
            except (requests.RequestException) as e:
                print(f"Error fetching data for CIK {CIK_code} ({item[2].upper()}): {e}")
                counter+=1
                continue
            extracted = extract_data(data)
            out[item[2].upper()] = extracted
            print(f"Processed {len(out)+counter} / {len(tickers)}")
    

    return out
df = get_more_data(screen["symbol"].tolist())


Processing 178 CIK data...
Processed 1 / 178
Processed 2 / 178
Processed 3 / 178
Processed 4 / 178
Processed 5 / 178
Processed 6 / 178
Processed 7 / 178
Processed 8 / 178
Processed 9 / 178
Processed 10 / 178
Processed 11 / 178
Processed 12 / 178
Processed 13 / 178
Processed 14 / 178
Processed 15 / 178
Processed 16 / 178
Processed 17 / 178
Processed 18 / 178
Processed 19 / 178
Processed 20 / 178
Processed 21 / 178
Processed 22 / 178
Processed 23 / 178
Processed 24 / 178
Error fetching data for CIK 0001388141 (EDD): 404 Client Error: Not Found for url: https://data.sec.gov/api/xbrl/companyfacts/CIK0001388141.json
Processed 25 / 178
Processed 26 / 178
Error fetching data for CIK 0000910068 (HIO): 404 Client Error: Not Found for url: https://data.sec.gov/api/xbrl/companyfacts/CIK0000910068.json
Processed 27 / 178
Processed 28 / 178
Processed 29 / 178
Processed 30 / 178
Processed 31 / 178
Processed 32 / 178
Processed 33 / 178
Processed 34 / 178
Processed 35 / 178
Processed 36 / 178
Processe

In [35]:
import pandas as pd
import numpy as np

def json_to_df(edgar_json: dict) -> pd.DataFrame:
    rows = []

    for ticker, metrics in edgar_json.items():
        row = {"ticker": ticker}

        # helper: parse list of (end, val) safely
        def _sorted_vals(key):
            vals = metrics.get(key, [])
            # keep only pairs with end and val
            vals = [(d, v) for d, v in vals if d is not None and v is not None]
            return sorted(vals, key=lambda x: x[0])

        # helper: record_date = latest end date across selected keys
        record_dates = []
        for key in ["assets", "liabilities", "cash", "shares", "long_term_debt",
                    "revenue", "net_income", "ebit", "operating_cash_flow"]:
            vals = _sorted_vals(key)
            if vals:
                record_dates.append(vals[-1][0])
        row["record_date"] = max(record_dates) if record_dates else np.nan

        # ---- STOCK METRICS: take latest ----
        for key in ["assets", "liabilities", "cash", "shares", "long_term_debt"]:
            vals = _sorted_vals(key)
            row[key] = vals[-1][1] if vals else np.nan

        # ---- FLOW METRICS: TTM if possible else latest ----
        for key in ["revenue", "net_income", "ebit", "operating_cash_flow"]:
            vals = _sorted_vals(key)
            if len(vals) >= 4:
                row[key] = sum(v for _, v in vals[-4:])
            elif vals:
                row[key] = vals[-1][1]
            else:
                row[key] = np.nan

        # ---- YoY comparisons for 3 important flow metrics ----
        # Simple: compare latest value vs a value ~1 year earlier (within a tolerance window)
        def _yoy(key):
            vals = _sorted_vals(key)
            if len(vals) < 2:
                return np.nan

            # convert date strings to pandas datetime for comparisons
            dfv = pd.DataFrame(vals, columns=["end", "val"])
            dfv["end"] = pd.to_datetime(dfv["end"], errors="coerce")
            dfv["val"] = pd.to_numeric(dfv["val"], errors="coerce")
            dfv = dfv.dropna(subset=["end", "val"]).sort_values("end")
            if len(dfv) < 2:
                return np.nan

            latest_end = dfv["end"].iloc[-1]
            latest_val = dfv["val"].iloc[-1]

            # target ~1 year before
            target = latest_end - pd.Timedelta(days=365)
            dfv["diff_days"] = (dfv["end"] - target).abs().dt.days

            # pick closest within a loose window (handles annual-only or irregular reporting)
            candidate = dfv.iloc[:-1].sort_values("diff_days").head(1)
            if candidate.empty:
                return np.nan
            if candidate["diff_days"].iloc[0] > 430:  # too far away
                return np.nan

            prev_val = candidate["val"].iloc[0]
            if prev_val == 0 or not np.isfinite(prev_val) or not np.isfinite(latest_val):
                return np.nan

            return (latest_val - prev_val) / abs(prev_val)

        row["assets_yoy"] = _yoy("assets")
        row["net_income_yoy"] = _yoy("net_income")
        row["operating_cash_flow_yoy"] = _yoy("operating_cash_flow")

        rows.append(row)

    return pd.DataFrame(rows)

edgar_df = json_to_df(df)

In [37]:
edgar_df.to_csv("lowcap_sec_data.csv", index = False)

In [38]:
edgar_df

,ticker,record_date,assets,liabilities,cash,shares,long_term_debt,revenue,net_income,ebit,operating_cash_flow,assets_yoy,net_income_yoy,operating_cash_flow_yoy
0,AUTL,2025-11-11,6.619470e+08,3.964950e+08,86124000.0,266143286.0,NaN,3.089100e+07,-442312000.0,-457373000.0,-646427000.0,-0.200054,0.036251,-0.284596
1,PEPG,2025-11-05,1.900570e+08,2.691700e+07,142775000.0,68748224.0,NaN,NaN,NaN,-171746000.0,-217623000.0,0.116446,NaN,-0.045996
2,TOI,2025-11-06,1.636190e+08,1.758930e+08,200000.0,98381340.0,NaN,NaN,-101711000.0,-69563000.0,-43998000.0,-0.086861,-0.041367,0.094519
3,RCKT,2025-10-31,3.680330e+08,5.436400e+07,75948000.0,108222228.0,NaN,NaN,-430089000.0,-446717000.0,-525442000.0,-0.065166,0.245612,0.046737
4,ILPT,2025-10-24,5.218846e+09,4.305216e+09,83173000.0,66659235.0,4.196825e+09,NaN,-336484000.0,157028000.0,95608000.0,-0.043258,0.114066,2.402956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,OPAD,2025-10-27,2.234610e+08,1.836090e+08,30959000.0,36859946.0,NaN,1.067703e+09,NaN,-59901000.0,3318000.0,-0.339826,NaN,2.314657
165,IBIO,2025-11-10,6.415500e+07,8.116000e+06,28111000.0,22487308.0,NaN,1.500000e+05,-19318000.0,-35383000.0,-37323000.0,1.616756,-0.421409,-0.524899
166,SOGP,2025-02-28,7.143700e+07,4.207700e+07,60534000.0,987114810.0,NaN,1.219591e+09,-34797000.0,-45162000.0,-6696000.0,-0.104799,0.465609,0.780130
167,BDSX,2025-10-28,8.872200e+07,9.044400e+07,16604000.0,7955685.0,5.001000e+07,1.395060e+08,NaN,-61531000.0,-96470000.0,-0.136467,NaN,0.460216
